In [ ]:
# Célula 1: Passo 2 - Carregamento de Dados e Data Augmentation (VERSÃO CORRIGIDA)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path

print(f"TensorFlow Versão: {tf.__version__}")

# --- 1. Definir Constantes e Caminhos ---
IMG_SIZE = (224, 224) 
BATCH_SIZE = 32

data_dir = Path('../data/casting_data')
train_dir = data_dir / 'train'
test_dir = data_dir / 'test'

if not train_dir.exists() or not test_dir.exists():
    print(f"Erro: Pastas de dados não encontradas.")
    print(f"Verifique se a pasta 'casting_data' está em '{data_dir.parent}'")
else:
    print(f"Pasta de treino encontrada: {train_dir}")
    print(f"Pasta de teste encontrada: {test_dir}")

# --- 2. Carregar Dados de Treino e Validação ---
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
print(f"\nClasses encontradas: {class_names}") # ['def_front', 'ok_front']

# --- 3. Camada de Data Augmentation (separada) ---
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ],
    name="data_augmentation"
)

# --- 4. Otimizar os Pipelines de Dados (COM A CORREÇÃO) ---
AUTOTUNE = tf.data.AUTOTUNE
# Importar a função de pré-processamento
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

def configure_for_performance(ds, augment=False, repeat=False):
    ds = ds.cache()
    if augment:
        # Aplicar a augmentation SÓ no treino
        ds = ds.map(lambda x, y: (data_augmentation(x), y), 
                    num_parallel_calls=AUTOTUNE)
    
    # Aplicar o pré-processamento (normalização [-1, 1]) EM TODOS
    # Esta é a correção: mover a camada problemática para cá.
    ds = ds.map(lambda x, y: (preprocess_input(x), y), 
                num_parallel_calls=AUTOTUNE)
    
    # CORREÇÃO: Adicionar .repeat() para o dataset de treino
    if repeat:
        ds = ds.repeat() # Isso corrige o 'UserWarning' do model.fit
    
    ds = ds.prefetch(buffer_size=AUTOTUNE) 
    return ds

# Aplicar a configuração
train_ds = configure_for_performance(train_ds, augment=True, repeat=True) # repeat=True
val_ds = configure_for_performance(val_ds, augment=False, repeat=False)

# --- 5. Preparar o Dataset de Teste ---
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False # Importante para a avaliação
)
# Aplicar a configuração no teste (sem augmentation, mas COM pré-processamento)
test_ds = configure_for_performance(test_ds, augment=False, repeat=False)

print("\nPipelines de dados (train_ds, val_ds, test_ds) criados e otimizados.")
print("O pré-processamento (normalização) foi movido para o pipeline de dados.")
print("Passo 2 concluído com sucesso!")

# (Opcional) Visualizar imagens
print("\nVisualizando 9 imagens de treino (já pré-processadas e aumentadas):")
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        # Reverter o pré-processamento (de [-1, 1] para [0, 1]) para visualização
        img = (images[i].numpy() + 1) / 2 
        plt.imshow(img)
        plt.title(f"{class_names[labels[i]]}")
        plt.axis("off")
plt.suptitle("Exemplo de Imagens de Treino (Pós-Augmentation e Pós-Processamento)")
plt.show()

In [ ]:
# Célula 2: Passo 3 - Modelagem (Transfer Learning) e Passo 4 - Treinamento (VERSÃO CORRIGIDA v2)

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
import numpy as np # Certifique-se de que numpy está importado
from pathlib import Path # Certifique-se de que Path está importado

print("Iniciando Passo 3: Construção do Modelo (Transfer Learning)...")

# --- 3.1: Carregar o Modelo Base (MobileNetV2) ---
# (Assumindo que IMG_SIZE foi definido na Célula 1 como (224, 224))
base_model = MobileNetV2(
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
    include_top=False, 
    weights='imagenet'
)

# --- 3.2: Congelar o Modelo Base ---
base_model.trainable = False
print(f"Modelo base (MobileNetV2) carregado e congelado.")

# --- 3.3: Criar a "Cabeça" de Classificação ---
inputs = keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = base_model(inputs, training=False) 
x = layers.GlobalAveragePooling2D()(x) 
x = layers.Dropout(0.2)(x) 
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

# --- 3.4: Compilar o Modelo ---
model.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

print("\nModelo final construído e compilado (SEM camadas de pré-processamento).")
model.summary() 

print("\nIniciando Passo 4: Treinamento do Modelo...")

# --- 4.1: Treinar ---
initial_epochs = 20

# --- CORREÇÃO: Usar .jpeg no glob e verificar caminhos ---
# (Assumindo que train_dir e BATCH_SIZE foram definidos na Célula 1)
try:
    # Contar imagens .jpeg
    train_image_files = list(train_dir.glob('*/*.jpeg'))
    num_train_images = len(train_image_files)
    
    if num_train_images == 0:
        print(f"AVISO: Nenhuma imagem .jpeg encontrada em {train_dir} e suas subpastas.")
        # Tentar .jpg como fallback, caso a extensão seja diferente
        train_image_files_jpg = list(train_dir.glob('*/*.jpg'))
        if len(train_image_files_jpg) > 0:
             print("Encontradas imagens .jpg. Usando essa contagem.")
             num_train_images = len(train_image_files_jpg)
        else:
             print("AVISO: Nenhuma imagem .jpg encontrada também.")

    print(f"Total de imagens de treino encontradas: {num_train_images}")

    steps_per_epoch = int(np.ceil(num_train_images * 0.8 / BATCH_SIZE))
    
    # Calcular validation_steps baseado no número total de imagens no diretório de treino
    # (Não precisamos re-glob, usamos o num_train_images que já temos)
    validation_steps = int(np.ceil(num_train_images * 0.2 / BATCH_SIZE))

    print(f"Calculando steps: {steps_per_epoch} steps por época (treino)")
    print(f"Calculando steps: {validation_steps} steps por época (validação)")

    # Verificar se os datasets train_ds e val_ds existem (da Célula 1)
    if 'train_ds' not in locals() or 'val_ds' not in locals():
        raise NameError("Datasets 'train_ds' ou 'val_ds' não encontrados. Execute a Célula 1.")

    history = model.fit(
        train_ds,
        epochs=initial_epochs,
        steps_per_epoch=steps_per_epoch, 
        validation_data=val_ds,
        validation_steps=validation_steps 
    )

    print("\nTreinamento concluído.")

    # --- 4.2: Salvar o Modelo Treinado ---
    models_dir = Path('../models')
    models_dir.mkdir(exist_ok=True) 
    model_path = models_dir / 'quality_control_model.h5'
    model.save(model_path)
    print(f"Modelo (corrigido) salvo com sucesso em: {model_path}")

    # --- 4.3: Visualizar o Histórico de Treinamento ---
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(acc, label='Acurácia (Treino)')
    plt.plot(val_acc, label='Acurácia (Validação)')
    plt.legend(loc='lower right')
    plt.title('Acurácia do Treinamento e Validação')
    plt.xlabel('Época')
    plt.ylabel('Acurácia')

    plt.subplot(1, 2, 2)
    plt.plot(loss, label='Perda (Treino)')
    plt.plot(val_loss, label='Perda (Validação)')
    plt.legend(loc='upper right')
    plt.title('Perda (Loss) do Treinamento e Validação')
    plt.xlabel('Época')
    plt.ylabel('Perda')
    plt.suptitle('Histórico do Treinamento (Modelo Corrigido)')
    plt.show()

    print("\nPassos 3 e 4 concluídos com sucesso!")

except NameError as ne:
    print(f"\nErro de Variável: {ne}")
    print("Certifique-se de que as variáveis IMG_SIZE, BATCH_SIZE, train_dir, train_ds, val_ds foram definidas na Célula 1.")
except Exception as e:
    print(f"\nOcorreu um erro inesperado: {e}")

In [ ]:
# Célula 3: Passo 5 - Avaliação no Conjunto de Teste (VERSÃO CORRIGIDA)

from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

print("Iniciando Passo 5 (Correção): Avaliação no Conjunto de Teste...")

# Verificar se as variáveis necessárias (da Célula 1 e 2) estão na memória
if 'test_ds' not in locals() or 'model' not in locals() or 'class_names' not in locals():
    print("Erro: Variáveis não encontradas.")
    print("Por favor, execute as Células 1 e 2 (corrigidas) primeiro.")
else:
    # --- 5.1: Avaliação Final (Acurácia) ---
    # Vamos avaliar o 'model' que ainda está na memória da Célula 2
    
    print("Avaliando o modelo treinado no conjunto de teste...")
    # 'test_ds' (da Célula 1) já está pré-processado, assim como o 'model' (da Célula 2) espera
    loss, accuracy = model.evaluate(test_ds) 

    print("\n--- Resultados da Avaliação no Conjunto de Teste ---")
    print(f"  Acurácia (Accuracy): {accuracy * 100:.2f}%")
    print(f"  Perda (Loss):        {loss:.4f}")
    print("--------------------------------------------------")

    # --- 5.2: Obter Predições e Rótulos Reais ---
    print("\nGerando predições para a Matriz de Confusão...")

    # Gerar predições para todo o conjunto de teste
    y_pred_probs = model.predict(test_ds)
    # Converter probabilidades (sigmoid) para classes (0 ou 1)
    y_pred = (y_pred_probs > 0.5).astype("int32").flatten()

    # Extrair os rótulos verdadeiros do dataset de teste
    y_true = np.concatenate([y for x, y in test_ds], axis=0)

    # --- AQUI ESTÁ A CORREÇÃO ---
    # Usar a variável 'class_names' que definimos na Célula 1
    target_names = class_names 
    print(f"Classes (da Célula 1): {target_names}") # ['def_front', 'ok_front']

    # --- 5.3: Relatório de Classificação e Matriz de Confusão ---

    print("\n--- Relatório de Classificação (Classification Report) ---")
    report = classification_report(y_true, y_pred, target_names=target_names)
    print(report)

    print("\n--- Matriz de Confusão ---")
    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(7, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=target_names, 
                yticklabels=target_names)
    plt.title('Matriz de Confusão (Dados de Teste)')
    plt.ylabel('Rótulo Verdadeiro (Real)')
    plt.xlabel('Rótulo Previsto (Modelo)')
    plt.show()

    print("\nPasso 5 concluído com sucesso!")

In [ ]:
# Célula 4: Passo 5 - Avaliação (VERSÃO CORRIGIDA)

from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

print("Iniciando Passo 5 (Correção): Avaliação no Conjunto de Teste...")

# Verificar se as variáveis necessárias (da Célula 1 e 2) estão na memória
if 'test_ds' not in locals() or 'model' not in locals() or 'class_names' not in locals():
    print("Erro: Variáveis não encontradas.")
    print("Por favor, execute as Células 1 e 2 primeiro.")
else:
    # --- 5.2: Obter Predições e Rótulos Reais ---
    # (O model.predict e a acurácia da célula anterior já funcionaram)
    
    print("Gerando predições para a Matriz de Confusão...")

    # Obter as predições (saídas 'sigmoid' entre 0 e 1)
    y_pred_probs = model.predict(test_ds)
    
    # Converter probabilidades para classes (0 ou 1)
    y_pred = (y_pred_probs > 0.5).astype("int32").flatten()

    # Obter os rótulos verdadeiros (y_true) do test_ds
    y_true = np.concatenate([y for x, y in test_ds], axis=0)

    # --- AQUI ESTÁ A CORREÇÃO ---
    # Usar a variável 'class_names' que definimos na Célula 1,
    # em vez de 'test_ds.class_names'
    target_names = class_names 
    print(f"Classes (da Célula 1): {target_names}")

    # --- 5.3: Relatório de Classificação e Matriz de Confusão ---

    print("\n--- Relatório de Classificação (Classification Report) ---")
    report = classification_report(y_true, y_pred, target_names=target_names)
    print(report)

    print("\n--- Matriz de Confusão ---")
    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(7, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=target_names, 
                yticklabels=target_names)
    plt.title('Matriz de Confusão (Dados de Teste)')
    plt.ylabel('Rótulo Verdadeiro (Real)')
    plt.xlabel('Rótulo Previsto (Modelo)')
    plt.show()

    print("\nPasso 5 concluído com sucesso!")